<h3>  &nbsp;&nbsp;Train on GPU&nbsp;&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a></h3>

1. On the main menu, click Runtime and select **Change runtime type**. Set "GPU" as the hardware accelerator.
1. Click Runtime again and select **Runtime > Run All**. You can also run the cells manually with Shift-ENTER. 

In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

In [2]:
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers 
import tensorflow.keras.optimizers as optimizers

from keras.datasets import cifar10
from keras.utils import to_categorical

import numpy as np

In [3]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# MODIFY THE CODE TO ADD A VALIDATION SET

validation_images = train_images[40000:]
validation_labels = train_labels[40000:]

train_images = train_images[:40000]
train_labels = train_labels[:40000]

In [4]:
# CHECKER:

print("Training set: images {:}\tlabels: {:}".format(train_images.shape, train_labels.shape ) )
print("validation set: images {:}\tlabels: {:}".format(validation_images.shape, validation_labels.shape ) )
print("Test set: images {:}\tlabels: {:}".format(test_images.shape, test_labels.shape ) )

Training set: images (40000, 32, 32, 3)	labels: (40000, 1)
validation set: images (10000, 32, 32, 3)	labels: (10000, 1)
Test set: images (10000, 32, 32, 3)	labels: (10000, 1)


In [5]:
train_images = train_images.reshape((40000, 32, 32, 3))
train_images = train_images.astype('float32') / 255

validation_images = validation_images.reshape((10000, 32, 32, 3))
validation_images = validation_images.astype('float32') / 255

test_images = test_images.reshape((10000, 32, 32, 3))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
validation_labels = to_categorical(validation_labels)
test_labels = to_categorical(test_labels)

In [6]:
print("Training set: images {:}\tlabels: {:}".format(train_images.shape, train_labels.shape ) )
print("validation set: images {:}\tlabels: {:}".format(validation_images.shape, validation_labels.shape ) )
print("Test set: images {:}\tlabels: {:}".format(test_images.shape, test_labels.shape ) )

Training set: images (40000, 32, 32, 3)	labels: (40000, 10)
validation set: images (10000, 32, 32, 3)	labels: (10000, 10)
Test set: images (10000, 32, 32, 3)	labels: (10000, 10)


In [7]:
from tensorflow.keras.applications import VGG19

conv_base = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(32, 32, 3))

conv_base.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
______________________________________________

## Features extraction

In [8]:
def extract_features(input_data):
    return conv_base.predict(input_data) 

train_features = extract_features(train_images)
validation_features = extract_features(validation_images)
test_features = extract_features(test_images)

In [9]:
print("train features:", train_features.shape)
print("validation features:", validation_features.shape)
print("test features:", test_features.shape)

train features: (40000, 1, 1, 512)
validation features: (10000, 1, 1, 512)
test features: (10000, 1, 1, 512)


In [10]:
train_features = np.reshape(train_features, (len(train_labels) , 512))
validation_features = np.reshape(validation_features, (len(validation_labels),512))
test_features = np.reshape(test_features, (len(test_labels), 512))

In [11]:
print("train features:", train_features.shape)
print("validation features:", validation_features.shape)
print("test features:", test_features.shape)

train features: (40000, 512)
validation features: (10000, 512)
test features: (10000, 512)


In [13]:
model = models.Sequential()

model.add(layers.Dense(512, activation='relu', input_dim= 512))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))


model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(10, activation='softmax'))


model.compile(optimizer=optimizers.RMSprop(lr=5e-5),
              loss='categorical_crossentropy',
                  metrics=['accuracy'])

model.load_weights('cifar10_VGG16.h5')

In [14]:
test_loss, test_acc = model.evaluate(test_features, test_labels)
print('test_acc:', test_acc)
test_loss

10000/10000 [==============================] - 1s 65us/sample - loss: 1.3550 - acc: 0.5874
test_acc: 0.5874


1.3549811477661133